In [1]:
#!/usr/bin/python -W ignore::DeprecationWarning

import sys
import pickle
sys.path.append("../tools/")
from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data
import pandas as pd
import sys
import pickle
import csv
import matplotlib.pyplot as plt

sys.path.append("../tools/")
from feature_format import featureFormat, targetFeatureSplit
#from poi_data import *
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit

from numpy import mean

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

from sklearn.metrics import accuracy_score, precision_score, recall_score


In [ ]:
#% ## Task 1: Select what features you'll use.
### features_list is a list of strings, each of which is a feature name.
### The first feature must be "poi".
#features_list = ['poi','salary'] # You will need to use more features

############# Task 1: Select what features you'll use.############


In [2]:
target_label = 'poi'

email_features_list = [
    'from_messages',
    'from_poi_to_this_person',
    'from_this_person_to_poi',
    'shared_receipt_with_poi',
    'to_messages',
    ]
    
financial_features_list = [
    'bonus',
    'deferral_payments',
    'deferred_income',
    'director_fees',
    'exercised_stock_options',
    'expenses',
    'loan_advances',
    'long_term_incentive',
    'other',
    'restricted_stock',
    'restricted_stock_deferred',
    'salary',
    'total_payments',
    'total_stock_value',
]

features_list = [target_label] + financial_features_list + email_features_list

### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "rb") as data_file:
    data_dict = pickle.load(data_file)

In [3]:
import numpy as np
df = pd.read_pickle('final_project_dataset.pkl')
df=pd.DataFrame.from_dict(df)
# df.pivot()
# df.melt(id_vars=features_list[1:])
df=df.T
df = pd.DataFrame(df)
# print(df.head(1))
print(type(df))
df.replace(to_replace='NaN', value=np.nan, inplace=True)


# df=df.replace(to_replace=np.nan, value=0)
dataf=df.fillna(0)
# df=df.to_dict
outliers = ['TOTAL', 'THE TRAVEL AGENCY IN THE PARK', 'LOCKHART EUGENE E']
for i in outliers:
    dataf=dataf.drop(i,axis=0)
dataf

<class 'pandas.core.frame.DataFrame'>


,salary,to_messages,deferral_payments,total_payments,loan_advances,bonus,email_address,restricted_stock_deferred,deferred_income,total_stock_value,...,from_poi_to_this_person,exercised_stock_options,from_messages,other,from_this_person_to_poi,poi,long_term_incentive,shared_receipt_with_poi,restricted_stock,director_fees
METTS MARK,365788.0,807.0,0.0,1061827.0,0.0,600000.0,mark.metts@enron.com,0.0,0.0,585062.0,...,38.0,0.0,29.0,1740.0,1.0,False,0.0,702.0,585062.0,0.0
BAXTER JOHN C,267102.0,0.0,1295738.0,5634343.0,0.0,1200000.0,0,0.0,-1386055.0,10623258.0,...,0.0,6680544.0,0.0,2660303.0,0.0,False,1586055.0,0.0,3942714.0,0.0
ELLIOTT STEVEN,170941.0,0.0,0.0,211725.0,0.0,350000.0,steven.elliott@enron.com,0.0,-400729.0,6678735.0,...,0.0,4890344.0,0.0,12961.0,0.0,False,0.0,0.0,1788391.0,0.0
CORDES WILLIAM R,0.0,764.0,0.0,0.0,0.0,0.0,bill.cordes@enron.com,0.0,0.0,1038185.0,...,10.0,651850.0,12.0,0.0,0.0,False,0.0,58.0,386335.0,0.0
HANNON KEVIN P,243293.0,1045.0,0.0,288682.0,0.0,1500000.0,kevin.hannon@enron.com,0.0,-3117011.0,6391065.0,...,32.0,5538001.0,32.0,11350.0,21.0,True,1617011.0,1035.0,853064.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GRAMM WENDY L,0.0,0.0,0.0,119292.0,0.0,0.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,False,0.0,0.0,0.0,119292.0
CAUSEY RICHARD A,415189.0,1892.0,0.0,1868758.0,0.0,1000000.0,richard.causey@enron.com,0.0,-235000.0,2502063.0,...,58.0,0.0,49.0,307895.0,12.0,True,350000.0,1585.0,2502063.0,0.0
TAYLOR MITCHELL S,265214.0,533.0,227449.0,1092663.0,0.0,600000.0,mitchell.taylor@enron.com,0.0,0.0,3745048.0,...,0.0,3181250.0,29.0,0.0,0.0,False,0.0,300.0,563798.0,0.0
DONAHUE JR JEFFREY M,278601.0,865.0,0.0,875760.0,0.0,800000.0,jeff.donahue@enron.com,0.0,-300000.0,1080988.0,...,188.0,765920.0,22.0,891.0,11.0,False,0.0,772.0,315068.0,0.0


In [4]:
data_dict

{'METTS MARK': {'salary': 365788,
  'to_messages': 807,
  'deferral_payments': 'NaN',
  'total_payments': 1061827,
  'loan_advances': 'NaN',
  'bonus': 600000,
  'email_address': 'mark.metts@enron.com',
  'restricted_stock_deferred': 'NaN',
  'deferred_income': 'NaN',
  'total_stock_value': 585062,
  'expenses': 94299,
  'from_poi_to_this_person': 38,
  'exercised_stock_options': 'NaN',
  'from_messages': 29,
  'other': 1740,
  'from_this_person_to_poi': 1,
  'poi': False,
  'long_term_incentive': 'NaN',
  'shared_receipt_with_poi': 702,
  'restricted_stock': 585062,
  'director_fees': 'NaN'},
 'BAXTER JOHN C': {'salary': 267102,
  'to_messages': 'NaN',
  'deferral_payments': 1295738,
  'total_payments': 5634343,
  'loan_advances': 'NaN',
  'bonus': 1200000,
  'email_address': 'NaN',
  'restricted_stock_deferred': 'NaN',
  'deferred_income': -1386055,
  'total_stock_value': 10623258,
  'expenses': 11200,
  'from_poi_to_this_person': 'NaN',
  'exercised_stock_options': 6680544,
  'from_

In [5]:
fieldnames = ['name'] + data_dict.itervalues().next().keys()

AttributeError: 'dict' object has no attribute 'itervalues'

In [ ]:
data_dict

In [ ]:
salary = [] 
for i in data_dict:
#     print(data_dict[i]['salary'])
    salary.append(data_dict[i]['salary'])
data_dict['METTS MARK']['salary']
salary
plt.hist(salary)
plt.gca().set(title='Frequency Histogram', ylabel='Frequency')

In [ ]:
def val_to_list(x):
#     x = [ i for i, a in locals().items() if a == x][0]
#     from_poi_to_this_person = [] 
    lcl = locals()
    lcl[x] = []
    for i in data_dict:
    #     print(data_dict[i]['salary'])
        x.append(data_dict[i][x])
    data_dict['METTS MARK'][x]
    print(x)
val_to_list('from_poi_to_this_person')
    

# from_poi_to_this_person = [] 
# for i in data_dict:
# #     print(data_dict[i]['salary'])
#     salary.append(data_dict[i]['from_poi_to_this_person'])
# data_dict['METTS MARK']['from_poi_to_this_person']
# from_poi_to_this_person

# from_this_person_to_poi = [] 
# for i in data_dict:
# #     print(data_dict[i]['salary'])
#     salary.append(data_dict[i]['from_this_person_to_poi'])
# data_dict['METTS MARK']['from_this_person_to_poi']

# to_messages = [] 
# for i in data_dict:
# #     print(data_dict[i]['salary'])
#     salary.append(data_dict[i]['to_messages'])
# data_dict['METTS MARK']['to_messages']

# 'from_messages',
#     'shared_receipt_with_poi',
#     'to_messages',



# from scipy.stats import pearsonr
# # calculate Pearson's correlation
# corr, _ = pearsonr(data1, data2)
# print('Pearsons correlation: %.3f' % corr)

In [ ]:
### 1.1.0 Explore csv file 
def make_csv(data_dict):
    """ generates a csv file from a data set"""
    fieldnames = ['name'] + data_dict.itervalues().next().keys()
    with open('data.csv', 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for record in data_dict:
            person = data_dict[record]
            person['name'] = record
            assert set(person.keys()) == set(fieldnames)
            writer.writerow(person)

In [ ]:
### 1.1.1 Dataset Exploration
print('# Exploratory Data Analysis #')
data_dict.keys()
print('Total number of data points: %d' % len(data_dict.keys()))
num_poi = 0
for name in data_dict.keys():
    if data_dict[name]['poi'] == True:
        num_poi += 1
print('Number of Persons of Interest: %d' % num_poi)
print('Number of people without Person of Interest label: %d' % (len(data_dict.keys()) - num_poi))


In [ ]:
data_dict['ALLEN PHILLIP K'].keys()

In [ ]:
###1.1.2 Feature Exploration
all_features = data_dict['ALLEN PHILLIP K'].keys()
print('Each person has %d features available' %  len(all_features))
### Evaluate dataset for completeness
missing_values = {}
for feature in all_features:
    missing_values[feature] = 0
for person in data_dict.keys():
    records = 0
    for feature in all_features:
        if data_dict[person][feature] == 'NaN':
            missing_values[feature] += 1
        else:
            records += 1

### Print results of completeness analysis
print('Number of Missing Values for Each Feature:')
ordered_list=[]
for feature in all_features:
    ordered_list.append(feature)
ordered_list.sort()
for feature in ordered_list:
    print("%s: %d" % (feature, missing_values[feature]))

#%%
# print(data_dict)
for person in data_dict.keys():
    records = 0
    for feature in all_features:
#         prev_value = None
        if data_dict[person][feature] == 'NaN':
            data_dict[person][feature]=0
# print(data_dict)



In [ ]:
ordered_list=[]
for feature in all_features:
    ordered_list.append(feature)
ordered_list.sort()
for feature in ordered_list:
    print("%s: %d" % (feature, 100*missing_values[feature]/len(data_dict)))
    
    
# for feature in all_features:
#     print("%s: %f" % (feature, )

In [ ]:
len(data_dict)
missing_values

In [ ]:
data_dict['TOTAL']

In [ ]:
################# Task 2: Remove outliers #####################

def PlotOutlier(data_dict, feature_x, feature_y):
    """ Plot with flag = True in Red """
    data = featureFormat(data_dict, [feature_x, feature_y, 'poi'])
    for point in data:
        x = point[0]
        y = point[1]
        poi = point[2]
        if poi:
            color = 'red'
        else:
            color = 'blue'
        plt.scatter(x, y, color=color)
    plt.xlabel(feature_x)
    plt.ylabel(feature_y)
    plt.show()

# 2.1 Visualise outliers
print(PlotOutlier(data_dict, 'total_payments', 'total_stock_value'))
print(PlotOutlier(data_dict, 'from_poi_to_this_person', 'from_this_person_to_poi'))
print(PlotOutlier(data_dict, 'salary', 'bonus'))
#Remove outlier TOTAL line in pickle file.
data_dict.pop( 'TOTAL', 0 )

# 2.2 Function to remove outliers
def remove_outlier(dict_object, keys):
    """ removes list of outliers keys from dict object """
    for key in keys:
        dict_object.pop(key, 0)

outliers = ['TOTAL', 'THE TRAVEL AGENCY IN THE PARK', 'LOCKHART EUGENE E']
remove_outlier(data_dict, outliers)


#%%


In [ ]:
PlotOutlier(data_dict, 'salary', 'bonus')

In [ ]:
#Making dataframes in pandas one for the keys and one for the values

df_values = pd.DataFrame.from_records(list(data_dict.values()))
print(df_values.head())

df_persons = pd.Series(list(data_dict.keys()))
print(df_persons.head())

#We see lot's of NaN values. So we have to take care of that. We will convert them to numpy nan and then to zero.
df_values.replace(to_replace='NaN', value=np.nan, inplace=True)

# Count number of NaN's for columns
print (df_values.isnull().sum())

# DataFrame dimension
print (df_values.shape)


df_null = df_values.replace(to_replace=np.nan, value=0)
df_null = df_values.fillna(0).copy(deep=True)
df_null.columns = list(df_values.columns.values)
print (df_null.isnull().sum())
print (df_null.head())

df_null.describe()

In [ ]:
plt.hist(data_dict.keys.values('salary'))
plt.gca().set(title='Frequency Histogram', ylabel='Frequency');

In [ ]:
################ Task 3: Create new feature(s) ####################

# 3.1 create new copies of dataset for grading
fraction_to_poi = [] 
for i in data_dict:
    fraction_to_poi.append(data_dict[i]['fraction_to_poi'])
dataf["fraction_to_poi"]=fraction_to_poi

fraction_from_poi = [] 
for i in data_dict:
    fraction_from_poi.append(data_dict[i]['fraction_from_poi'])
dataf["fraction_from_poi"]=fraction_from_poi

dataf.head(2)

data = featureFormat(dataf, features_list)
labels, features = targetFeatureSplit(data)
labels
# # 3.3 create new copies of feature list for grading
# my_feature_list = features_list +['to_messages', 'from_poi_to_this_person', 'from_messages', 'from_this_person_to_poi','shared_receipt_with_poi', 'fraction_to_poi']

# # my_feature_list = 

# # 3.4 get K-best features
# num_features = 10

# # 3.5 functio using SelectKBest
# def get_k_best(data_dict, features_list, k):
#     """ runs scikit-learn's SelectKBest feature selection
#         returns dict where keys=features, values=scores
#     """
#     data = featureFormat(data_dict, features_list)
#     labels, features = targetFeatureSplit(data)

#     k_best = SelectKBest(k=k)
#     k_best.fit(features, labels)
#     scores = k_best.scores_
#     print(scores)
#     unsorted_pairs = zip(features_list[1:], scores)
#     sorted_pairs = list(reversed(sorted(unsorted_pairs, key=lambda x: x[1])))
#     k_best_features = dict(sorted_pairs[:k])
#     print ("{0} best features: {1}\n".format(k, k_best_features.keys(), scores))
#     return k_best_features


# best_features = get_k_best(my_dataset, my_feature_list, num_features)

# test=["fraction_to_poi",'shared_receipt_with_poi',"fraction_from_poi","salary",'exercised_stock_options','total_stock_value','bonus']

# my_feature_list = [target_label] + test#list(set(best_features.keys()))

# # 3.6 print features
# # print ("{0} selected features: {1}\n".format(len(my_feature_list) - 1, my_feature_list[1:]))
# print ("{0} selected features: {1}\n".format(len(my_feature_list), best_features))

# # 3.7 extract the features specified in features_list
# data = featureFormat(my_dataset, my_feature_list,sort_keys = True)
# # split into labels and features
# labels, features = targetFeatureSplit(data)

# # 3.8 scale features via min-max
# from sklearn import preprocessing
# scaler = preprocessing.MinMaxScaler()
# features = scaler.fit_transform(features)

# #%%


In [ ]:

################ Task 3: Create new feature(s) ####################

# 3.1 create new copies of dataset for grading
my_dataset = data_dict

## 3.2 add new features to dataset
def compute_fraction(poi_messages, all_messages):
    """ return fraction of messages from/to that person to/from POI"""    
    if poi_messages == 0 or all_messages == 0:
        return 0.
    fraction = poi_messages / all_messages
    return fraction

for name in my_dataset:
    data_point = my_dataset[name]
    from_poi_to_this_person = data_point["from_poi_to_this_person"]
    to_messages = data_point["to_messages"]
    fraction_from_poi = compute_fraction(from_poi_to_this_person, to_messages)
    data_point["fraction_from_poi"] = fraction_from_poi
    from_this_person_to_poi = data_point["from_this_person_to_poi"]
    from_messages = data_point["from_messages"]
    fraction_to_poi = compute_fraction(from_this_person_to_poi, from_messages)
    data_point["fraction_to_poi"] = fraction_to_poi

# 3.3 create new copies of feature list for grading
my_feature_list = features_list +['to_messages', 'from_poi_to_this_person', 'from_messages', 'from_this_person_to_poi','shared_receipt_with_poi', 'fraction_to_poi']

# my_feature_list = 

# 3.4 get K-best features
num_features = 10

# 3.5 functio using SelectKBest
def get_k_best(data_dict, features_list, k):
    """ runs scikit-learn's SelectKBest feature selection
        returns dict where keys=features, values=scores
    """
    data = featureFormat(data_dict, features_list)
    labels, features = targetFeatureSplit(data)

    k_best = SelectKBest(k=k)
    k_best.fit(features, labels)
    scores = k_best.scores_
    print(scores)
    unsorted_pairs = zip(features_list[1:], scores)
    sorted_pairs = list(reversed(sorted(unsorted_pairs, key=lambda x: x[1])))
    k_best_features = dict(sorted_pairs[:k])
    print ("{0} best features: {1}\n".format(k, k_best_features.keys(), scores))
    return k_best_features


best_features = get_k_best(my_dataset, my_feature_list, num_features)

test=["fraction_to_poi",'shared_receipt_with_poi',"fraction_from_poi"]#,"salary",'exercised_stock_options','total_stock_value','bonus']

my_feature_list = [target_label] +ltest#list(set(best_features.keys()))

# 3.6 print features
# print ("{0} selected features: {1}\n".format(len(my_feature_list) - 1, my_feature_list[1:]))
print ("{0} selected features: {1}\n".format(len(my_feature_list), best_features))

# 3.7 extract the features specified in features_list
data = featureFormat(my_dataset, my_feature_list,sort_keys = True)
# split into labels and features
labels, features = targetFeatureSplit(data)

# 3.8 scale features via min-max
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
features = scaler.fit_transform(features)

#%%


In [ ]:
### Task 4: Try a varity of classifiers
### Please name your classifier clf for easy export below.
### Note that if you want to do PCA or other multi-stage operations,
### you'll need to use Pipelines. For more info:
### http://scikit-learn.org/stable/modules/pipeline.html

# Provided to give you a starting point. Try a variety of classifiers.

##########################Task 4: Using algorithm########################

###4.1  Gaussian Naive Bayes Classifier
from sklearn.naive_bayes import GaussianNB
g_clf = GaussianNB()

###4.2  Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression

l_clf = Pipeline(steps=[
        ('scaler', StandardScaler()),
        ('classifier', LogisticRegression(C=1e-08, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, 
max_iter=100, multi_class='ovr', penalty='l2', random_state=42, solver='liblinear', tol=0.001, verbose=0))])

###4.3  K-means Clustering
from sklearn.cluster import KMeans
k_clf = KMeans(n_clusters=2, tol=0.001)


###4.4 Support Vector Machine Classifier
from sklearn.svm import SVC
s_clf = SVC(kernel='rbf', C=1000,gamma = 0.0001,random_state = 42, class_weight = 'balanced')

###4.5 Random Forest
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(max_depth = 5,max_features = 'sqrt',n_estimators = 10, random_state = 42)


###4.6 Gradient Boosting Classifier
from sklearn.ensemble  import GradientBoostingClassifier
gb_clf = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=100,random_state = 42)

###4.7 evaluate function
def evaluate_clf(clf, features, labels, num_iters=1000, test_size=0.3):
    print (clf)
    accuracy = []
    precision = []
    recall = []
    first = True
    for trial in range(num_iters):
        features_train, features_test, labels_train, labels_test =\
            train_test_split(features, labels, test_size=test_size)
        clf.fit(features_train, labels_train)
        predictions = clf.predict(features_test)
        accuracy.append(accuracy_score(labels_test, predictions))
        precision.append(precision_score(labels_test, predictions))
        recall.append(recall_score(labels_test, predictions))
        if trial % 10 == 0:
            if first:
                sys.stdout.write('\nProcessing')
            sys.stdout.write('.')
            sys.stdout.flush()
            first = False

    print ("done.\n")
    print ("precision: {}".format(mean(precision)))
    print ("recall:    {}".format(mean(recall)))
    return mean(precision), mean(recall)


### 4.8 Evaluate all functions
evaluate_clf(g_clf, features, labels)


In [ ]:
evaluate_clf(l_clf, features, labels)

In [ ]:
evaluate_clf(k_clf, features, labels)

In [ ]:
evaluate_clf(s_clf, features, labels)

In [ ]:
evaluate_clf(rf_clf, features, labels)

In [ ]:
evaluate_clf(gb_clf, features, labels)

In [ ]:
### Select Logistic Regression as final algorithm
clf = l_clf


# dump your classifier, dataset and features_list so
# anyone can run/check your results

pickle.dump(clf, open("../final_project/my_classifier.pkl", "wb"))
pickle.dump(my_dataset, open("../final_project/my_dataset.pkl", "wb"))
pickle.dump(my_feature_list, open("../final_project/my_feature_list.pkl", "wb"))


#%%

In [ ]:
### Task 5: Tune your classifier to achieve better than .42 precision and recall 
### using our testing script. Check the tester.py script in the final project
### folder for details on the evaluation method, especially the test_classifier
### function. Because of the small size of the dataset, the script uses
### stratified shuffle split cross validation. For more info: 
### http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.StratifiedShuffleSplit.html

# Example starting point. Try investigating other evaluation techniques!

features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=0.3, random_state=42)

#%%


In [ ]:
### Task 6: Dump your classifier, dataset, and features_list so anyone can
### check your results. You do not need to change anything below, but make sure
### that the version of poi_id.py that you submit can be run on its own and
### generates the necessary .pkl files for validating your results.

dump_classifier_and_data(clf, my_dataset, features_list)


In [ ]:
data_dict